In [13]:
import numpy as np
import pandas as pd

In [14]:
file_name = 'supply_chain_logisitcs_problem.xlsx'
df_freight = pd.read_excel(file_name, sheet_name='FreightRates')
df_order = pd.read_excel(file_name, sheet_name='OrderList')
df_plant_ports = pd.read_excel(file_name, sheet_name='PlantPorts')
df_prod_per_plant = pd.read_excel(file_name, sheet_name='ProductsPerPlant')
df_vmi_cust = pd.read_excel(file_name, sheet_name='VmiCustomers')
df_wh_cap = pd.read_excel(file_name, sheet_name='WhCapacities')
df_wh_costs = pd.read_excel(file_name, sheet_name='WhCosts')

In [15]:
# display(df_freight.head(3))
# display(df_order.head(3))
# display(df_plant_ports.head(3))
# display(df_prod_per_plant.head(3))
# display(df_vmi_cust.head(3))
# display(df_wh_cap.head(3))
# display(df_wh_costs.head(3)) 

In [6]:
df = pd.merge(df_order[['Order Date','Order ID','Carrier','Origin Port','Destination Port','Unit quantity','Weight']], df_plant_ports, left_on = ['Origin Port'], right_on = ['Port']).rename(columns = {'Plant Code' : 'origin_plant_code'}).drop(['Port'], axis = 1)
df = pd.merge(df, df_plant_ports, left_on = ['Destination Port'], right_on = ['Port']).rename(columns = {'Plant Code' : 'destination_plant_code'}).drop(['Port'], axis = 1)
df = pd.merge(df, df_freight, left_on = ['Carrier', 'Origin Port', 'Destination Port'], right_on = ['Carrier', 'orig_port_cd', 'dest_port_cd'])
df.head()

,Order Date,Order ID,Carrier,Origin Port,Destination Port,Unit quantity,Weight,origin_plant_code,destination_plant_code,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,svc_cd,minimum cost,rate,mode_dsc,tpt_day_cnt,Carrier type
0,2013-05-26,1.447385e+09,V444_0,PORT09,PORT09,332,1.023143,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTP,31.2784,12.2784,GROUND,0,V88888888_0
1,2013-05-26,1.447385e+09,V444_0,PORT09,PORT09,332,1.023143,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTD,31.2784,13.2784,GROUND,0,V88888888_0
2,2013-05-26,1.447338e+09,V444_0,PORT09,PORT09,358,0.613886,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTP,31.2784,12.2784,GROUND,0,V88888888_0
3,2013-05-26,1.447338e+09,V444_0,PORT09,PORT09,358,0.613886,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTD,31.2784,13.2784,GROUND,0,V88888888_0
4,2013-05-26,1.447407e+09,V444_0,PORT09,PORT09,409,37.041956,PLANT16,PLANT16,PORT09,PORT09,0.0,5000.0,DTP,31.2784,12.2784,GROUND,0,V88888888_0


In [8]:
df_freight_grouped = df_freight.groupby(['Carrier','orig_port_cd','dest_port_cd']).agg({
    'minm_wgh_qty' : lambda x : sum(x)/len(x),
    'max_wgh_qty' : lambda x : sum(x)/len(x),
    'rate' : lambda x : sum(x)/len(x),
    'minimum cost' : lambda x : sum(x)/len(x)
}).reset_index().rename(columns = {'minimum cost' : 'min_cost','minm_wgh_qty' : 'min_wgh_qty'})
display(df_freight_grouped)

,Carrier,orig_port_cd,dest_port_cd,min_wgh_qty,max_wgh_qty,rate,min_cost
0,V444_0,PORT02,PORT09,570.000000,20569.990000,0.084480,23.838400
1,V444_0,PORT03,PORT09,0.000000,5000.000000,12.694200,28.944200
2,V444_0,PORT04,PORT09,570.000000,20569.990000,0.052000,2.477200
3,V444_0,PORT09,PORT09,0.000000,5000.000000,12.778400,31.278400
4,V444_1,PORT02,PORT09,197.556000,1190.745000,0.291140,23.557340
5,V444_1,PORT04,PORT09,197.556000,1190.745000,0.242000,20.909780
6,V444_1,PORT05,PORT09,170.130000,1001.952500,0.457895,35.954091
7,V444_1,PORT06,PORT09,45.473493,111.272697,2.492247,6.917300
8,V444_1,PORT10,PORT09,78.945000,12570.433750,0.431400,12.350850
9,V444_2,PORT02,PORT09,570.000000,20569.990000,0.079040,9.027200


In [17]:
supply = {'PORT02': 2000, 'PORT03': 1500, 'PORT04': 3000, 'PORT09': 0}  # Origin port supply
demand = {'PORT09': 5000}  # Demand at destination
supply, demand

({'PORT02': 2000, 'PORT03': 1500, 'PORT04': 3000, 'PORT09': 0},
 {'PORT09': 5000})

In [ ]:
df_cost = df_freight_grouped.groupby(['orig_port_cd','dest_port_cd'])['min_cost'].min().unstack()
df_cost = round(df_cost, 0)
display(df_cost)
display(df_cost.index)
df_cost = df_cost[df_cost.index.isin(df['Origin Port'])]
display(df_cost)
array_cost_pulp = df_cost.to_numpy().tolist()
array_cost_pulp

dest_port_cd,PORT09
orig_port_cd,
PORT02,5.0
PORT03,25.0
PORT04,2.0
PORT05,8.0
PORT06,6.0
PORT07,19.0
PORT08,20.0
PORT09,24.0
PORT10,7.0


Index(['PORT02', 'PORT03', 'PORT04', 'PORT05', 'PORT06', 'PORT07', 'PORT08',
       'PORT09', 'PORT10', 'PORT11'],
      dtype='object', name='orig_port_cd')

dest_port_cd,PORT09
orig_port_cd,
PORT04,2.0
PORT09,24.0


[[2.0], [24.0]]

In [42]:
df_cost

dest_port_cd,PORT09
orig_port_cd,
PORT04,2.0
PORT09,24.0
